In [16]:
import pysb.bng
import kholodenko_basic as kb
#import kholodenko_m1 as m
#import kholodenko_m3 as m3 # this is just to test redundant rules
import sympy as sp
from sympy.printing import latex
import re
import numpy as np
import tabulate as tb
import pandas as pd
import collections
pysb.bng.__file__

'/anaconda/lib/python2.7/site-packages/pysb/bng.pyc'

In [2]:
pysb.bng.generate_equations(kb.model)
#pysb.bng.generate_equations(m.model)
#pysb.bng.generate_equations(m3.model)

In [23]:
"""for i, j in zip(kb.model.species, ke.model.species):
 
    assert i.is_equivalent_to(j)

species_map = dict(zip(range(len(kb.model.species)), kb.model.species))

ke_subs, kb_subs = [zip(m.model.expressions,[e.expand_expr(expand_observables=True) for e in m.model.expressions]) for m in ke, kb]
print ke.model.parameters[2]
ke_dict, kb_dict = [dict([((x['reactants'], x['products']), x['rate']) for x in y])
                                    for y in [ke.model.reactions, kb.model.reactions]]

"""
"""
TODO:
0. NOT DONE:
    organize by products and reactants then subset by rule name because rule names are NOT conserved
DONE:
1. Substitute the species __s0 with some identifier Done 23 oct
2. Find a list of all of the UNIQUE bidirectional reactions Done 23 oct"""



def fmt_rct(r, species_dict, fmt='plain'):
    
    arrow = {'latex': " $\longleftrightarrow$ ",
             'plain': "<-->"}[fmt]
    fmt_fct = {'latex': lambda x: "$%s$" % latex(x).replace("__", "").replace(" % ", ""),
               'plain': lambda x: str(x).replace(" % ", "")}[fmt]
    r_schema = " + ".join(map(fmt_fct, r.reactants)) + arrow + "".join(map(fmt_fct, r.products))
    return [r.name.replace("_", "-"), r_schema, fmt_fct(r.fwd_rate), fmt_fct(r.rev_rate), fmt_fct(r.kd)]



def gen_table(model_list, fmt='plain'):
    
    # this is assuming that all models must have the same species....
    m = model_list[0]
    species_dict = dict([('__s%d' % idx, str(m.model.species[idx])) for idx in range(len(m.model.species))])
    
    all_rates_tidy = pd.DataFrame(data=np.vstack([[[m.__name__.replace("_", "-")] + fmt_rct(x, species_dict, fmt=fmt) for x in gen_rct_rates(m)] for m in model_list]), columns=["model", "rule", "schema", "fwd_rate", "rev_rate", "kd"])

    #rxn_dict = { k: np.hstack([np.array([[k[0], 'K_f'], [k[1], 'K_r'], ["", 'K_d']]), 
    #                                                np.array([d[k] for d in dicts]).T]) for k in dicts[0] }
    #rxn_array = np.vstack([np.array([["Rule", "Rate"] + [mod.__name__.replace("_", "-") for mod in model_list]]), np.vstack(rxn_dict.values())])

    return np.array(species_dict.items()),  all_rates_tidy.sort_values(["schema"])
        
def gen_rct_rates(m):
    ## from a model, create a generator of all reaction rates binned by forward, reverse, kd
  
    rct_subs = dict(zip(m.model.expressions,[e.expand_expr(expand_observables=True) for e in m.model.expressions])) #\+ [(sp.Symbol('__s%d' % m.model.get_species_index(complex_pattern=species)), latex(species)) for species in m.model.species])
    species_map = dict(zip(range(len(m.model.species)), m.model.species))
    species = lambda tup: [species_map[x] for x in tup]
    
    rule_names = [r.name for r in m.model.rules]
    fwd_rxn = dict([(r["rule"][0], r) for r in m.model.reactions if not r["reverse"][0]])
    rev_rxn = dict([(r["rule"][0], r) for r in m.model.reactions if r["reverse"][0]])
    rxn_tup = collections.namedtuple("reaction", ["name", "reactants", "products", "fwd_rate", "rev_rate", "kd"])
    
    def make_rct_tuple(rule_name):
        
        rev_rate = extra_simple(rev_rxn[rule_name]["rate"].subs(rct_subs))
        fwd_rate = extra_simple(fwd_rxn[rule_name]["rate"].subs(rct_subs))
        kd = extra_simple(rev_rate/fwd_rate)
        reactants, products = [species(fwd_rxn[rule_name][s]) for s in ["reactants", "products"]]

        return rxn_tup(rule_name, reactants, products, fwd_rate, rev_rate, kd)
    
    return map(make_rct_tuple, rule_names)


def extra_simple(mul):
    return sp.powsimp((sp.expand_power_base(sp.powdenest(sp.logcombine(sp.expand_log(mul.simplify(), force=True), force=True), force=True), force=True)), force=True)

species, rxns = gen_table([kb, kb], fmt='latex')
with open('model.txt', 'w') as out:
    out.write(tb.tabulate(species))

    
def write_latex_dict_to_file(latex_table, outfile):
    with open(outfile, 'w') as out:
        out.write("\n".join(["\documentclass{article}",
                             "\usepackage[utf8]{inputenc}", 
                             "\usepackage{amsmath}", 
                             "\\begin{document}", 
                            '\def\\arraystretch{2}']))
        out.write(tb.tabulate(latex_table, tablefmt='latex_raw'))
        out.write("\end{document}")

write_latex_dict_to_file(rxns[["model", "rule", "fwd_rate", "rev_rate", "kd"]].as_matrix(), 'rxns.txt')
## other shit here
#def fmt_reaction(reaction, m=m):
    
#    reaction = str(reaction)
    #for species in range(len(m.model.species)):
    #    match = '__s' + str(species)
        #reaction = reaction.replace(match, '[' + str(m.model.species[species]).replace("(", "_{").replace(")", "}").replace(" % ", ":")+']')
        
     #   reaction = re.sub(r'\([^%]+\)', "", \
     #                     reaction.replace(match, "[%s]" % str(m.model.species[species]))).replace(" % ", "")
#    return reaction


def gen_latex(rct_rates, m):
    
    species_name_dict = dict([('__s%d' % idx, str(m.model.species[idx])) for idx in range(len(m.model.species))])
    
    def fmt_rxn(reaction):
        return latex(reaction)
    
    for r in rct_rates:
        rct_schema = " + ".join(map(fmt_reaction, r.reactants)) \
        + " $\longleftrightarrow$ " + "".join(map(fmt_reaction, r.products))
        rates = [r.fwd_rate, r.rev_rate, r.kd]
        yield ((r.name, rct_schema), map(lambda eq: "$%s$" % fmt_reaction(eq), rates)) # fmt_reaction(latex(eq))




def rct_rates(fwd_key, rct_dict, rct_subs):
    ## Takes keys of FORWARD reactions
    fwd_rate = extra_simple(rct_dict[fwd_key].subs(rct_subs).simplify())
    rev_rate = extra_simple(rct_dict[(fwd_key[1], fwd_key[0])].subs(rct_subs).simplify())
    k_d = extra_simple(rev_rate/fwd_rate)
    return (fwd_rate, rev_rate, k_d)

def extra_simple(mul):
    return sp.expand_power_base(sp.powdenest(sp.logcombine(sp.expand_log(mul.simplify(), force=True), force=True), force=True), force=True)

def latex_table_models(model_list):
    ## Takes in a list of models with the same k's, though calculated potentially differently
    dicts = [dict([x for x in gen_latex(gen_rct_rates(model))]) for model in model_list]
    print np.array([['K_f', 'K_r', 'K_d']]).shape
    latex_dict = { k: np.hstack([np.array([[k[0], 'K_f'], [k[1], 'K_r'], ["", 'K_d']]), 
                                                    np.array([d[k] for d in dicts]).T]) for k in dicts[0] }
    return np.vstack([np.array([["Rule", "Rate"] + [mod.__name__.replace("_", "-") for mod in model_list]]), np.vstack(latex_dict.values())])


                  
    
#write_latex_dict_to_file(latex_table_models([m3]), 'model.txt')


### dir(m)

In [ ]:
m3.model.reactions
m3.model.rules[0].name

In [ ]:
import collections
g = collections.namedtuple("reaction", ["h", "i", "j"])
dir(g(2, 3, 5))